In [ ]:
##import
from binance.client import Client
import pandas as pd
import os
from finlab import crypto
import talib
from binance.enums import *
import math
import time
from tqdm import tqdm
import threading
import requests
##顯示pandas所有行與列
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

##api設置
api_key=os.environ.get('binance_api')
api_secret=os.environ.get('binance_secret')
client = Client(api_key, api_secret)
line_UserID='U1f1b16cd25612228b144370f3d4e0b38'

dic_position={1:'多頭頭寸',0:'未開倉',-1:'空頭頭寸',2:'已執行階段停利策略，等待下次開倉'}
assets_detail_list=['free','locked','borrowed','interest','netAsset']

In [ ]:
### Function

## Set COHLV_data Function
def set_COHLV_data(Dataframe):
    COHLV={'Close':pd.Series(Dataframe['Close']),
            'High':pd.Series(Dataframe['High']),
            'Low':pd.Series(Dataframe['Low']),
            'Open':pd.Series(Dataframe['Open'])}
    return COHLV


## get_margin_assets_detail Function
def get_margin_assets_detail():
    margin_assets_info = client.get_margin_account() 
    margin_account_detail=pd.DataFrame()
    x=0
    for i in margin_assets_info['userAssets']:
        margin_account_detail=pd.concat([margin_account_detail,pd.DataFrame([i])],axis=0)
    margin_account_detail.set_index('asset', inplace = True)
    margin_account_detail.drop(margin_account_detail[(margin_account_detail.netAsset == '0')].index, inplace=True)
    return(margin_account_detail)

def set_loan_detail(margin_account_detail):
    try :
        loan_detail={'APE_max_loan_amount':client.get_max_margin_loan(asset='APE'),
                    'BUSD_max_loan_amount':client.get_max_margin_loan(asset='BUSD'),
                    'APE_current_loan':str(float(margin_account_detail['borrowed']['APE'])+float(margin_account_detail['interest']['APE'])) ,
                    'BUSD_current_loan':str(float(margin_account_detail['borrowed']['BUSD'])+float(margin_account_detail['interest']['BUSD'])),
                    }
    except:
        loan_detail={'APE_max_loan_amount':client.get_max_margin_loan(asset='APE'),
                    'BUSD_max_loan_amount':client.get_max_margin_loan(asset='BUSD')
                    }
    return(loan_detail)


def send_assets_detail():
    a=get_margin_assets_detail()
    message='您目前槓桿帳戶明細如下'
    for x in a.iloc:
        message+='\n'
        message+=x.name
        message+='\n'
        for y in assets_detail_list:
            message=message+y+':'+str(x[y])
            message+='\n'

    print(message)
    send_line(line_UserID,message)


## Strategy_function

def EMA_Strategy(COHLV_data,fast=14,slow=26):
#     ##計算EMA快慢線
    EMA_fast=talib.EMA(COHLV_data['Close'],fast)
    EMA_slow=talib.EMA(COHLV_data['Close'],slow)
#     ##判斷長部頭寸及短部頭寸
#     signal_long = (EMA_fast > EMA_slow) & (EMA_fast.shift() < EMA_slow.shift())
#     signal_short = (EMA_slow > EMA_fast) & (EMA_slow.shift() < EMA_fast.shift())
#     signal = EMA_fast-EMA_slow
    signal_long = (EMA_fast > EMA_slow) & (EMA_fast.shift() < EMA_slow.shift())
    signal_short = (EMA_slow > EMA_fast) & (EMA_slow.shift() < EMA_fast.shift())
    cross_signal = EMA_fast -EMA_slow
    signal = signal_long
    signal[signal_short] = -1
    signal = signal.astype(int)
    return [signal,cross_signal]

def KD_Strategy(COHLV_data,ftk=10,sk=2,sd=4):
    KD=talib.STOCH(COHLV_data['High'],
                COHLV_data['Low'],
                COHLV_data['Close'],
                fastk_period=ftk,
                slowk_period=sk,
                slowd_period=sd)
    signal = KD[0]-KD[1]
    signal_long = (KD[0] > KD[1]) & (KD[0].shift() < KD[1].shift())
    signal_short = (KD[1] > KD[0]) & (KD[1].shift() < KD[0].shift())
    signal = signal_long
    signal[signal_short] = -1
    signal = signal.astype(int)
    return signal
    
    
# UlTMATE PROJECT-Operation Arclight START Function
def set_interval(func, sec):
    def func_wrapper():
        set_interval(func, sec)
        func()
    t = threading.Timer(sec, func_wrapper)
    t.start()
    return 

def send_line(UserID,msg):
    requests.post('https://benson-bot-node.herokuapp.com/pushmsg',data={"userId":UserID,"msg":msg})

In [ ]:
def long_step1(loan_detail,margin_account_detail):
    order = client.create_margin_order(
        symbol='APEBUSD',
        side='BUY',
        type='MARKET',
        quantity=str(math.ceil(float(loan_detail['APE_current_loan']) * 1) / 1.0))
    
    print('已買入 {} APE，成交額 {}，成交均價 {}，訂單編號 {}'.format(order['fills'][0]['qty'],order['cummulativeQuoteQty'],order['fills'][0]['price'],order['orderId']))
    
    send_line(line_UserID,"Step1.買進空頭頭寸還款額 成功")
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~')
    time.sleep(3)     
    
def long_step2(loan_detail,margin_account_detail):

    transaction = client.repay_margin_loan(asset='APE', amount=loan_detail['APE_current_loan'])
    print('已償還所有 APE，流水編號{}'.format(transaction['tranId']))
    
    send_line(line_UserID,"Step2.償還空頭頭寸已欠款額 成功")
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~')
    time.sleep(3)

def long_step3(loan_detail,margin_account_detail):

    transaction = client.create_margin_loan(asset='BUSD', amount=loan_detail['BUSD_max_loan_amount']['amount'])
    print('已借入 BUSD 最大貸款金額，流水編號{}'.format(transaction['tranId']))
    
    send_line(line_UserID,"Step3.借入最大多頭頭寸 成功")
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~')
    time.sleep(3)
    
def long_step4(loan_detail,margin_account_detail):
    
    order = client.create_margin_order(
    symbol='APEBUSD',
    side='BUY',
    type='MARKET',
    quantity=str(math.ceil(math.floor(float(margin_account_detail['free']['BUSD'])/10) * 10 / COHLV['Close'][-1] * 100)/100)
    ##買進BUSD帳戶無條件捨去至十位數可買APE總量
    )
    print('已買入 {} APE，成交額 {}，成交均價 {}，訂單編號 {}'.format(order['fills'][0]['qty'],order['cummulativeQuoteQty'],order['fills'][0]['price'],order['orderId']))
    
    send_line(line_UserID,"Step4.建立多頭頭寸 成功")
    
def long_a_position():
    global try_time
    print('建立多頭頭寸 開始')

    print('Step1.買進空頭頭寸還款額')
    try_time=0
    running_step(long_step1)

    print('Step2.償還空頭頭寸已欠款額')
    try_time=0
    running_step(long_step2)

    print('Step3.借入最大多頭頭寸')
    try_time=0
    running_step(long_step3)

    print('Step4.建立多頭頭寸')
    try_time=0
    running_step(long_step4)
    

def short_step1(loan_detail,margin_account_detail):

    order = client.create_margin_order(
        symbol='APEBUSD',
        side='SELL',
        type='MARKET',
        quantity=str(math.floor(float(margin_account_detail['free']['APE'])*100)/100))
    print('已出售 {} APE，成交額 {}，成交均價 {}，訂單編號 {}'.format(order['fills'][0]['qty'],order['cummulativeQuoteQty'],order['fills'][0]['price'],order['orderId']))
    
    send_line(line_UserID,"Step1.出售多頭頭寸 成功")
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~')
    time.sleep(3)
    
def short_step2(loan_detail,margin_account_detail):

    transaction = client.repay_margin_loan(asset='BUSD', amount=loan_detail['BUSD_current_loan'])
    print('已償還所有 BUSD，流水編號{}'.format(transaction['tranId']))
    
    send_line(line_UserID,"Step2.償還多頭頭寸已欠款額 成功")
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~')
    time.sleep(3)
    
def short_step3(loan_detail,margin_account_detail):

    transaction = client.create_margin_loan(asset='APE', amount=loan_detail['APE_max_loan_amount']['amount'])
    print('已借入APE最大貸款金額，流水編號{}'.format(transaction['tranId']))
    
    send_line(line_UserID,"Step3.借入空頭頭寸款項 成功")
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~')
    time.sleep(3)
    
def short_step4(loan_detail,margin_account_detail):
    order = client.create_margin_order(
        symbol='APEBUSD',
        side='SELL',
        type='MARKET',
        quantity=str(math.floor(float(margin_account_detail['free']['APE'])*100)/100))
    print('已出售 {} APE，成交額 {}，成交均價 {}，訂單編號 {}'.format(order['fills'][0]['qty'],order['cummulativeQuoteQty'],order['fills'][0]['price'],order['orderId']))
    
    send_line(line_UserID,"Step4.建立空頭頭寸 成功")
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~')
    
def short_a_position():
    global try_time
    
    print('建立空頭頭寸 開始')

    print('Step1.出售多頭頭寸')
    try_time=0
    running_step(short_step1)

    print('Step2.償還多頭頭寸已欠款額')
    try_time=0
    running_step(short_step2)

    print('Step3.借入空頭頭寸款項')
    try_time=0
    running_step(short_step3)

    print('Step4.建立空頭頭寸')
    try_time=0
    running_step(short_step1)
    
    
def running_step(step_func):
    global try_time,loan_detail,margin_account_detail
    temp1 = get_margin_assets_detail()
    temp2 = set_loan_detail(get_margin_assets_detail())
    try:
        if try_time <3:
            temp1 = get_margin_assets_detail()
            temp2 = set_loan_detail(get_margin_assets_detail())
            step_func(loan_detail=temp2,margin_account_detail=temp1)
        else:
            print('請確認交易是否異常')
    except:
        temp1 = get_margin_assets_detail()
        temp2 = set_loan_detail(get_margin_assets_detail())
        try_time +=1
        print('失敗 已嘗試執行次數 {}/3'.format(try_time))
        with tqdm(total=2) as pbar:
            for i in range(2):
                time.sleep(1)
                pbar.update(1)
        running_step(step_func)
        
def close_a_position():
    global try_time
    if current_position == -1:
        
        print('Step1.買進空頭頭寸還款額')
        try_time=0
        running_step(long_step1)
        
        print('Step2.償還空頭頭寸已欠款額')
        try_time=0
        running_step(long_step2)
        
        print('平倉完成')
    if current_position == 1:
        print('Step1.出售多頭頭寸')
        try_time=0
        running_step(short_step1)
    
        print('Step2.償還多頭頭寸已欠款額')
        try_time=0
        running_step(short_step2)
        print('平倉完成')

In [ ]:
def set_current_equity():
    current_equity = str(math.floor(float(client.get_recent_trades(symbol='BTCBUSD')[0]['price'])*
                     float(client.get_margin_account()['totalNetAssetOfBtc'])*100)/100)
    return current_equity

def set_current_position():
    global current_position
    try:
        if 'current_position' in globals():
            None
        else:
            temp=int(input('\n請輸入您目前的倉位模式：  1.未開倉  2.多頭頭寸  3.空頭頭寸\n'))
            if temp==1:
                current_position=0
            if temp==2:
                current_position=1
            if temp==3:
                current_position=-1
            else:
                set_current_position()
    except:
        print('請輸入數字!')
        set_current_position()

def set_stop_profit_point(equity_percent_change):
    global stop_profit_point,highest_profit_point
    
    if 'stop_profit_point' in globals():
        None
    else:
        stop_profit_point = 0

    if 'highest_profit_point' in globals():
        None
    else:
        highest_profit_point = 0
        
    dic={0.05:0.00,
        0.08:0.05,
        0.12:0.06,
        0.15:0.07,
        0.18:0.08,
        0.2:0.1,
        0.25:0.12
        0.3:0.15}

    for i in dic.keys():
        if equity_percent_change > i:
            stop_profit_point=dic[i]

def reset_profit_point():
    global stop_profit_point,highest_profit_point
    stop_profit_point = 0
    highest_profit_point=0


In [ ]:
def Ultimate_Project_Operation_Arclight():
    
    time_start = time.time()
    ##取得APEBUSD最新資料(30分K)
    global margin_account_detail,loan_detail,COHLV,try_time,initial_equity,current_position
    print('----------------------------------------------------------------------------------------------------------------------------------')
    df = crypto.get_all_binance('APEBUSD', '30m')
    df = df.drop(['Close_time','Quote_av','Trades','Tb_base_av','Tb_quote_av','Ignore'],axis=1)
    ##設定COHLV變數
    COHLV=set_COHLV_data(df)

    ##print目前槓桿帳戶資料
    print('\n槓桿帳戶資產總覽(已隱藏為零資產)\n')
    margin_account_detail = get_margin_assets_detail()
    print(margin_account_detail)
    loan_detail = set_loan_detail(get_margin_assets_detail())
    set_current_position()
    print('\n您目前的倉位模式為 「{}」'.format(dic_position[current_position]))
    #設定策略啟動資金

    if 'initial_equity' in globals():
        None
    else:
        initial_equity = set_current_equity()

    current_equity = set_current_equity()
    equity_percent_change = float(current_equity)/float(initial_equity)-1

    print('\n★ 您目前帳戶權益為 ${} BUSD ★'.format(current_equity))
    print('★ 距上次執行策略時資產 {} USD , 變動率 {} ★'.format(initial_equity,equity_percent_change))

    set_stop_profit_point(equity_percent_change)

    if highest_profit_point == 0 and stop_profit_point == 0:
        print('\n目前策略未達階段停利策略')
    else:
        print('目前策略已達階段停利策略 {}/{}，目前變動率為 {}'.format(highest_profit_point,stop_profit_point,equity_percent_change))
    try:
        # set_strategy
        signal = EMA_Strategy(COHLV_data=COHLV)[0]
        cross_signal = EMA_Strategy(COHLV_data=COHLV)[1].fillna(0)
        print('\n執行Operation Arclight計畫')
        print('最近資料時間(UTC+0)：')
        print(signal.index[-1])

        if cross_signal[-1] > 0:
            print('▲ 多頭頭寸 ▲')
        elif cross_signal[-1] < 0:
            print('▼ 空頭頭寸 ▼')


        #----------------------Start----------------------
        #當信號交叉時，將曾達停利停損條件的倉位模式調整為未開倉
        if signal[0] !=0 and current_position == 2:
            current_position=0

        #當目前行情為空頭的時候，目前倉位為多頭時或未開倉時且未曾達停利停損條件，執行空頭頭寸
        if cross_signal[-1] > 0 and ((current_position == 0) or (current_position == -1)) and current_position!=2:
            long_a_position()
            current_position=1
            reset_profit_point()
            initial_equity = set_current_equity()
            send_line(line_UserID,"行情轉多，已為您建立多頭頭寸")
            send_assets_detail()
            print('已重設策略啟動資金，為{}'.format(current_equity))
            
        #當目前行情為多頭的時候，目前倉位為空頭或未開倉時且未曾達停利停損條件，執行多頭頭寸
        elif cross_signal[-1] < 0 and ((current_position == 0) or (current_position == 1)) and current_position!=2:
            short_a_position()
            current_position=-1
            reset_profit_point()
            initial_equity = set_current_equity()
            send_line(line_UserID,"行情轉多，已為您建立空頭頭寸")
            send_assets_detail()
            print('已重設策略啟動資金，為{}'.format(current_equity))
        #停利設定，階段式停利
        if highest_profit_point != 0 and stop_profit_point != 0:
            if equity_percent_change < stop_profit_point:
                print('已達階段式停利條件，開始平倉')
                close_position()
                current_position=2
                reset_profit_point
                initial_equity = set_current_equity()
                
                send_line(line_UserID,"已達階段式停利條件，已為您關閉所有倉位")
                send_assets_detail()
                print('已重設策略啟動資金，為{}'.format(current_equity))
                
        #-----------------------End----------------------
    except:
        print('策略語法有誤')
        
    print('本次執行總用時 {} 秒'.format(str(time.time() - time_start)))

In [ ]:
send_line(line_UserID,"您已開始執行策略")
send_assets_detail()
Ultimate_Project_Operation_Arclight()

In [ ]:
set_interval(Ultimate_Project_Operation_Arclight,10)